In [1]:
from util import badc_csv
import pandas as pd
from pathlib import Path

hallo


Define paths to header templates:

In [2]:
path_FaIR_header_general_info = 'badc_header_temp/badc_header_FaIR_model.csv'
path_FaIR_warming_header_general_info ='badc_header_temp/badc_header_FaIR_model_warming.csv'
path_FaIR_hist_header_general_info = 'badc_header_temp/badc_header_FaIR_model_hist.csv'


In [3]:
def redo_SSPs_to_badc_csv(path_orig, path_out, fp_global_default, fp_var_default):
    """
    :param path_orig: path to folder of original files
    :param path_out: path to put the output
    :param fp_global_default: path to header template for global 
    :param fp_var_default: path to header template for variables 
    :return: 
    """

    path_out.mkdir(exist_ok=True, parents=True)
    for f in list(path_orig.glob('*.csv')):
        filename = f.name

        f_out = path_out / filename
        #print(f_out)
        fn_parts =  filename.split('_')
        scenario_name = ' '.join(fn_parts[1:-1])
        print(scenario_name)
        # print('hallo')
        add_global_info = [['comments','G',f'Scenario: {scenario_name}'],]
        badc_csv.write_badc_header(
            f,
            f_out,
            add_global_info,
            
            #variable_dic,
            #read_csv_kwargs=None,
            fp_global_default,
            fp_var_default,
        )
    #print('done')
    return

In [4]:
from pathlib import Path

df = pd.read_csv('data_in/SSPs/ERF_ssp119_1750-2500.csv', index_col=0, header=None)
df.iloc[0,:]
df.index[0]
var_labs = list(df.iloc[0,:]) + [df.index[0]]
var_labs

In [5]:
path_in =  Path('data_in/SSPs/')
path_out = Path('data_badc_csv/SSPs/')
redo_SSPs_to_badc_csv(path_in, path_out, path_FaIR_header_general_info, path_FaIR_header_general_info)

# %%

ssp126 minorGHGs
year
HFC-134a
HFC-23
HFC-32
HFC-125
HFC-143a
HFC-152a
HFC-227ea
HFC-236fa
HFC-245fa
HFC-365mfc
HFC-43-10mee
NF3
SF6
SO2F2
CF4
C2F6
C3F8
c-C4F8
CFC-12
CFC-11
CFC-113
CFC-114
CFC-115
CFC-13
HCFC-22
HCFC-141b
HCFC-142b
CH3CCl3
CCl4
CH3Cl
CH3Br
CH2Cl2
CHCl3
Halon-1211
Halon-1301
Halon-2402
n-C4F10
n-C5F12
n-C6F14
i-C6F14
C7F16
C8F18
CFC-112
CFC-112a
CFC-113a
CFC-114a
HCFC-133a
HCFC-31
HCFC-124
ssp245 minorGHGs
year
HFC-134a
HFC-23
HFC-32
HFC-125
HFC-143a
HFC-152a
HFC-227ea
HFC-236fa
HFC-245fa
HFC-365mfc
HFC-43-10mee
NF3
SF6
SO2F2
CF4
C2F6
C3F8
c-C4F8
CFC-12
CFC-11
CFC-113
CFC-114
CFC-115
CFC-13
HCFC-22
HCFC-141b
HCFC-142b
CH3CCl3
CCl4
CH3Cl
CH3Br
CH2Cl2
CHCl3
Halon-1211
Halon-1301
Halon-2402
n-C4F10
n-C5F12
n-C6F14
i-C6F14
C7F16
C8F18
CFC-112
CFC-112a
CFC-113a
CFC-114a
HCFC-133a
HCFC-31
HCFC-124
ssp585 minorGHGs
year
HFC-134a
HFC-23
HFC-32
HFC-125
HFC-143a
HFC-152a
HFC-227ea
HFC-236fa
HFC-245fa
HFC-365mfc
HFC-43-10mee
NF3
SF6
SO2F2
CF4
C2F6
C3F8
c-C4F8
CFC-12
CFC-11
CFC-11

In [6]:
def make_badc_csvs_for_slcf_warming_ranges(
        fn_orig,# = (BASE_DIR /'data_in/chris_slcf_warming_ranges.csv'),
        fn_base,# = 'slcf_warming_ranges',
        path_out, # = (BASE_DIR /'data_in_badc_csv'/'slcf_warming_ranges'),
        path_FaIR_warming_header_general_info,
    default_unit = 'degrees C'
):
    """
    Redo slcf_warming_ranges
    :param path_FaIR_warming_header_general_info: header template for files
    :param fn_orig: original file
    :param fn_base: base to add file name specifics to
    :param path_out: directory for output
    :return:
    """


    path_out.mkdir(parents=True, exist_ok=True)


    df_orig = pd.read_csv(fn_orig,index_col=0)
    scenario_ls = df_orig.loc[:,'scenario'].unique()
    dic_scn_data = dict()
    if len(df_orig['base_period'].unique())>1:
        print('ERROR: Different base_periods found! ')
        sys.exit()
    base_per = df_orig['base_period'].unique()[0]
    df_orig = df_orig.drop(['base_period'], axis=1)
    pcts =['p05','p16','p50','p84','p95']

    for scn in scenario_ls:
        dic_scn_data[scn] = {}
        df_sub_scn = df_orig[df_orig['scenario']==scn]
        df_sub_scn = df_sub_scn.drop('scenario', axis=1)
        for p in pcts:
            df_sub_scn_p = df_sub_scn[['year','forcing',p]]#.set_index('year')
            df_n = df_sub_scn_p.set_index(['forcing','year']).unstack()[p].T

            fn_sub = fn_base + f'_{scn}_{p}.csv'
            fp = path_out / fn_sub
            df_n.to_csv(fp)
            print(scn)
            add_global_info = [['comments','G',f'Scenario: {scn}, percentile: {p}'],]
            badc_csv.write_badc_header(
                fp,
                fp,
                add_global_info,
                #variable_dic,
                #read_csv_kwargs=None,
                path_FaIR_warming_header_general_info,
                path_FaIR_warming_header_general_info,
                #fp_var_default = path_FaIR_header_general_info
                default_unit=default_unit
            )

        #    df_sub_scn_p = df_sub_scn
    return

In [7]:
path_in_wr = Path('data_in/slcf_warming_ranges.csv')
fn_base = 'slcf_warming_ranges'
path_out = Path('data_badc_csv/slcf_warming_ranges/')


In [8]:
make_badc_csvs_for_slcf_warming_ranges(
        path_in_wr,# = (BASE_DIR /'data_in/chris_slcf_warming_ranges.csv'),
        fn_base,# = 'slcf_warming_ranges',
        path_out, # = (BASE_DIR /'data_in_badc_csv'/'slcf_warming_ranges'),
        path_FaIR_warming_header_general_info,
    
)

ssp119
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp119
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp119
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp119
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp119
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp126
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp126
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp126
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp126
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp126
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp245
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp245
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp245
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp245
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp245
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp334
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp334
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp334
year
aerosol
anthro
bc_on_snow
ch4
hfc
o3
slcf
ssp334
year
aerosol
anthro
b